In [ ]:
import os
import cv2
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [ ]:
def read_dicom_image(image_file, voi_lut=True, fix_monochrome=True):
    """
    Reads a dicom image from a file an returns a numpy array.
    References: https://www.kaggle.com/trungthanhnguyen0502/eda-vinbigdata-chest-x-ray-abnormalities
    Args:
        image_file:
        voi_lut:
        fix_monochrome:

    Returns:

    """
    dicom = pydicom.read_file(image_file)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
train_images = []
for root, subdirs, files in os.walk('directory'):
    for file in files:
        train_images.append(os.path.join(root, file))

In [ ]:
images = []
for path in tqdm(train_images):
    img = read_dicom_image(path)
    resized = cv2.resize(img, (224,224))
    images.append(resized)

In [ ]:
images_npy = np.stack(images)

In [ ]:
plt.imshow(images_npy[2,:,:], cmap="gray")

In [ ]:
np.save('BrixIA224.npy', images_npy)

In [ ]:
del images_npy

In [ ]:
images_npy = np.load('BrixIA224.npy')

In [ ]:
plt.imshow(images_npy[2,:,:], cmap="gray")